In [1]:
import io
import torch
import numpy as np
from transformers import AutoProcessor, PaliGemmaForConditionalGeneration
from PIL import Image
from datasets import load_dataset
from datasets import Image as HFImage
from anls_star import anls_score
from fastprogress.fastprogress import progress_bar
from torchvision.transforms import ToPILImage, PILToTensor
from walloc import walloc
class Config: pass

In [2]:
ds = load_dataset("danjacobellis/docvqa",split='test')
model_id = "google/paligemma-3b-ft-docvqa-448"
model = PaliGemmaForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="cuda",
    revision="bfloat16",
)
processor = AutoProcessor.from_pretrained(model_id)

Resolving data files:   0%|          | 0/57 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/57 [00:00<?, ?it/s]

/home/dgj335/.local/lib/python3.10/site-packages/transformers/models/paligemma/configuration_paligemma.py:137: FutureWarning: The `vocab_size` attribute is deprecated and will be removed in v4.44, Please use `text_config.vocab_size` instead.
  warnings.warn(
`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
def compute_score(item,res):
    prompt = item['question']
    image = item['image'].resize((res, res), resample=Image.Resampling.LANCZOS).convert("RGB")
    model_inputs = processor(text=prompt, images=image, return_tensors="pt").to(model.device)
    input_len = model_inputs["input_ids"].shape[-1]
    with torch.inference_mode():
        generation = model.generate(**model_inputs, max_new_tokens=100, do_sample=False)
        generation = generation[0][input_len:]
        pred = processor.decode(generation, skip_special_tokens=True)
    score, _ = anls_score(item['answer'], pred, return_gt=True)
    return score, pred

In [4]:
scores = []
preds = []
qid = []
res = 448
for item in progress_bar(ds):
    score, pred = compute_score(item,res)
    scores.append(score)
    preds.append(pred)
    qid.append(item['questionId'])

/home/dgj335/.local/lib/python3.10/site-packages/anls_star/anls_star.py:513: UserWarning: Treating ground truth as a list of options. This is a compatibility mode for ST-VQA-like datasets.
  warnings.warn(


9953 MiB

In [5]:
5188 / (10*60+56)

7.908536585365853

In [ ]:
ds = ds.add_column('preds',preds)

In [ ]:
ds.push_to_hub("danjacobellis/docvqa_paligemma224",split='test')